In [75]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.


In [76]:
import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor


In [77]:
#//SETUP//

folder_path = "/Users/71635/Documents/Computer Science/Sigma Xi/Ml model Meteor Showers/CAMS-Summary"
output_file = "combined_meteor_data.csv"

#converting all .txt files into .csv files 
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        old_path = os.path.join(folder_path, filename)
        new_path = os.path.join(folder_path, filename.replace(".txt", ".csv"))
        os.rename(old_path, new_path)
        print(f"Renamed {filename} -> {filename.replace('.txt', '.csv')}")


all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

In [78]:
 
 #//COMBINING ALL OF THE DATA INTO ONE DATAFRAME//
with open(output_file, 'w') as f_out:
    first_file = True
    for file in all_files:
        print(f"Processing: {file}")
        chunk = pd.read_csv(file, chunksize=50000)  
        for sub_chunk in chunk:
            sub_chunk.dropna(inplace=True)  
            sub_chunk.drop_duplicates(inplace=True)  
            sub_chunk.to_csv(f_out, mode='a', header=first_file, index=False)
            first_file = False  

Processing: /Users/71635/Documents/Computer Science/Sigma Xi/Ml model Meteor Showers/CAMS-Summary/2017_05_23_00_00_00_CAMS_MEAS_00094.csv
Processing: /Users/71635/Documents/Computer Science/Sigma Xi/Ml model Meteor Showers/CAMS-Summary/2017_08_14_00_00_00_CAMS_MEAS_00382.csv
Processing: /Users/71635/Documents/Computer Science/Sigma Xi/Ml model Meteor Showers/CAMS-Summary/2017_09_20_00_00_00_CAMS_MEAS_00128.csv
Processing: /Users/71635/Documents/Computer Science/Sigma Xi/Ml model Meteor Showers/CAMS-Summary/2017_05_09_00_00_00_CAMS_MEAS_00081.csv
Processing: /Users/71635/Documents/Computer Science/Sigma Xi/Ml model Meteor Showers/CAMS-Summary/2017_06_17_00_00_00_CAMS_MEAS_00052.csv
Processing: /Users/71635/Documents/Computer Science/Sigma Xi/Ml model Meteor Showers/CAMS-Summary/2017_06_19_00_00_00_CAMS_MEAS_00091.csv
Processing: /Users/71635/Documents/Computer Science/Sigma Xi/Ml model Meteor Showers/CAMS-Summary/2017_09_24_00_00_00_CAMS_MEAS_00027.csv
Processing: /Users/71635/Documents

In [79]:
#//CLEANING//

df = pd.read_csv("combined_meteor_data.csv") #load master data
df.dropna(inplace = True) #remove rows w missing values
df.drop_duplicates(inplace=True) #remove duplicate rows 
df.reset_index(drop=True, inplace=True) #resets index 
df.to_csv("cleaned_meteor_data.csv", index=False) #save the cleaned data 

column_names = ["frame", "time", "range", "height", "vel", "internr", "mv_vmag", "mv_flux", "lat", "long", "azims", "zenang"] #defines column names
df = pd.read_csv("cleaned_meteor_data.csv", names=column_names, header=None) #load the dataset and add column names
df.to_csv("cleaned_meteor_data.csv", index=False) #saves 

In [80]:
df["time"] = pd.to_datetime(df["time"], errors = "coerce") #convert time into datetime format
#gets the hour, minute and second from the time column, making them their own columns
df["hour"] = df["time"].dt.hour
df["minute"] = df["time"].dt.minute
df["second"] = df["time"].dt.second
df.drop(columns=["time"], inplace = True) #drops the "time" column
#normalize numerical columns
scaler = MinMaxScaler()
num_cols = ["range", "height", "vel", "internr", "mv_vmag", "mv_flux", "lat", "long", "azims", "zenang", "hour", "minute", "second"]
df[num_cols] = scaler.fit_transform(df[num_cols])


/Users/71635/miniconda3/lib/python3.12/site-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/Users/71635/miniconda3/lib/python3.12/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [81]:
#//SPLIT INTO TEST AND TRAINING SETS// 
X = df.drop(columns = ["lat", "long", "height", "vel"]) #input
Y = df[["lat", "long", "height", "vel"]] #output

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 21)


In [82]:
#//TRAINING THE MODEL// 
'''
#convert the column "frame" to numeric
X_train["frame"] = pd.to_numeric(X_train["frame"], errors="coerce")
X_test["frame"] = pd.to_numeric(X_test["frame"], errors="coerce")
'''

#drops the values (NaN) that cannot be converted
X_train.dropna(inplace=True)
X_test.dropna(inplace=True)


# Drop "frame" instead of converting
X_train.drop(columns=["frame"], inplace=True)
X_test.drop(columns=["frame"], inplace=True)

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=21)
model.fit(X_train, Y_train)



ValueError: Found array with 0 sample(s) (shape=(0, 9)) while a minimum of 1 is required by RandomForestRegressor.

In [ ]:
#//SPLIT INTO TEST AND TRAINING SETS 
'''
df = pd.read_csv("cleaned_meteor_data.csv")
#selects relevant values/columns
X = df[['time', 'range', 'height', 'vel', 'mv_vmag', 'mv_flux', 'lat', 'long', 'azims', 'zenang']]
y = df[['lat', 'long', 'height']] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #split into training (80%) and testing (20%) sets

print("horay")
'''

'\ndf = pd.read_csv("cleaned_meteor_data.csv")\n#selects relevant values/columns\nX = df[[\'time\', \'range\', \'height\', \'vel\', \'mv_vmag\', \'mv_flux\', \'lat\', \'long\', \'azims\', \'zenang\']]\ny = df[[\'lat\', \'long\', \'height\']] \n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #split into training (80%) and testing (20%) sets\n\nprint("horay")\n'